In [1]:
import io, os, sys, ssl, time, json, socket, struct, random, datetime, picamera, threading
import repl, serial, pigpio

server_ip = 'yzlab3.chem.nyu.edu'
img_port = 54321
ctrl_port = 54322 
sensor_port = 54324

server_cert_self_signed  = 'secret/server.crt' 
client_certs_self_signed = 'secret/rover.crt'
client_key_self_signed   = 'secret/rover.key'

# Establish connection with server
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert_self_signed)
context.load_cert_chain(certfile=client_certs_self_signed, keyfile=client_key_self_signed)

In [2]:
def chksum_nmea(sentence): 
    csum = 0  
    for c in sentence: 
        csum ^= ord(c)
    return str(hex(csum)).upper()[2:] 

def newGPRMC():  
    lat, lon = 4043.831, 7359.70
    for i in range(36,1000):
        lat = lat + 0.003*random.randint(-50,20) 
        lon = lon + 0.003*random.randint(-20,100) 
        timestamp = time.strftime("%H%M%S", time.gmtime())
        chksum = chksum_nmea('GPRMC,{},A,{},N,0{},W,022.4,084.4,210719,003.1,W'.format(timestamp,lat,lon))  
        yield '$GPRMC,{},A,{},N,0{},W,022.4,084.4,210719,003.1,W*{}'.format(timestamp,lat,lon,chksum)
GPRMC = newGPRMC() # gps_simulator, when I need to test tracking system

In [3]:
class SENSOR(): 
    def __init__(self):
        self.data = {}
        self.Temperature = 0 
        self.interval = 3
        self.sg = serial.Serial(port='/dev/ttyS0', baudrate=9600, timeout=3) # serial commu with gps chip
        sm = serial.Serial(port='/dev/ttyACM0', baudrate=115200, parity=serial.PARITY_NONE, stopbits=serial.STOPBITS_ONE, timeout=0) # serial commu with microbit
        self.repl = repl.REPL(sm) 
        self.repl.to_raw()
        threading.Thread(target=self.reporter).start()
        time.sleep(3) # wait for the readiness of sensors 
    
    def reporter(self): 
        while True:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sensor_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            sensor_ss.connect((server_ip, sensor_port))
            while True:
                data = {}
                while True:
                    data['P'] = GPRMC.__next__()
                    break
                    if gprmc[0:6] == b'$GPRMC': data['P'] = self.sg.readline() # You can get GPS data only if the on-board LED blinks.  
                self.repl.send_command("print(temperature())") 
                self.Temperature = self.repl.wait_response().rstrip()
                self.repl.send_command("print(accelerometer.get_values())") 
                data['a'] = self.repl.wait_response()
                self.repl.send_command("print(compass.heading())") 
                data['N'] = self.repl.wait_response()
                j = json.dumps(data)
                try:
                    sensor_ss.send(struct.pack('<H', len(j)))
                    sensor_ss.send(j.encode()) 
                except Exception as e:
                    sensor_ss.shutdown(socket.SHUT_RDWR) 
                    sensor_ss.close()
                    print('sensor channel closes and will reopen in 15 seconds', e)
                    time.sleep(15)
                    break   
                time.sleep(self.interval)    

sensor = SENSOR() 

In [4]:
camera = picamera.PiCamera(resolution=(640, 480)) 
camera.rotation = 270
camera.start_preview() 
time.sleep(2)

# https://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream
# https://picamera.readthedocs.io/en/release-1.13/recipes2.html#web-streaming

def imgd():
    while True:
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            img_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            img_ss.connect((server_ip, img_port)) 
            img_io = img_ss.makefile('wb')
            stream = io.BytesIO()
            for foo in camera.capture_continuous(stream, 'jpeg'): 
                dt = datetime.datetime.now().strftime('@ %H:%M:%S %b %d %Y')
                camera.annotate_text = f'Celsius:{sensor.Temperature.rstrip()} {dt}'  
                img_io.write(struct.pack('<L', stream.tell()))
                img_io.flush()
                stream.seek(0)
                img_io.write(stream.read()) 
                img_io.flush()
                stream.seek(0)
                stream.truncate()
                time.sleep(1)
        except Exception as e:
            print(e)
            time.sleep(15)
    
threading.Thread(target=imgd).start() 

In [5]:
!sudo pkill pigpiod
!sudo pigpiod -p 8889 ### Start PiGPIO Server / Deamon          http://abyz.me.uk/rpi/pigpio/pigpiod.html
pi = pigpio.pi(port=8889) ### Start PiGPIO Client
servo = lambda l1, l2: [pi.set_servo_pulsewidth(_1, _2) for _1, _2 in zip(l1, l2)]
motor = lambda l1, l2: [pi.set_PWM_dutycycle(   _1, _2) for _1, _2 in zip(l1, l2)]
drive_pin, reverse_pin      = 16, 20
elevation_pin, rotation_pin = 19, 26  # {sky: 1300 ~ 1700 ~ 2300 & front: 500 ~ 2500}
top_pin, bottom_pin         =  6, 13  # {top:900 ~ 1800 & bottom: 1500 ~ 2350}


def ctrd():
    l = struct.calcsize('<Hi')
    while True:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ctrl_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
        ctrl_ss.connect((server_ip
                         , ctrl_port))
        while True:
            try:
                code, value = struct.unpack('<Hi', ctrl_ss.recv(l)) 
            except Exception as e:
                ctrl_ss.shutdown(socket.SHUT_RDWR) 
                ctrl_ss.close()
                print('Control channel closes and will reopen in 15 seconds', e)
                time.sleep(15)
                break 
            if   code ==  0: servo([top_pin, bottom_pin], [value, 3300-value]) 
            elif code ==  2: pi.set_servo_pulsewidth(rotation_pin, value) 
            elif code ==  5: pi.set_servo_pulsewidth(elevation_pin, value) 
            elif code == 10: motor([drive_pin, reverse_pin], [0, value])  # break
            elif code ==  9: motor([reverse_pin, drive_pin], [0, value])  # gas
            elif code ==304 and value: servo([top_pin, bottom_pin], [1350, 1950]) 
threading.Thread(target=ctrd).start()